# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 25, 20, 15

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:47<00:00,  8.40s/it]


,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
19,PriceChange,20,91.67,79.17,91.67,83.33,79.17,79.17,83.33,91.67,83.33,91.67,83.33,79.17,79.17,83.33,87.50,83.33,75.00,83.33,79.17,83.33,83.33,83.33,75.00,75.00,-1.82,-1.74,-2.23,-1.99,-1.63,-2.12,-2.09,-2.14,-2.09,-2.73,-2.98,-2.73,-2.48,-2.45,-2.70,-2.38,-1.59,-1.69,-2.37,-2.42,-2.25,-2.44,-2.27,-2.80,24,82.812500,307.500000
16,PriceChange,17,95.24,76.19,90.48,85.71,80.95,80.95,85.71,90.48,85.71,90.48,80.95,76.19,76.19,80.95,85.71,80.95,76.19,85.71,80.95,85.71,85.71,85.71,76.19,76.19,-1.86,-1.23,-2.19,-1.96,-1.63,-2.07,-2.06,-1.96,-2.00,-2.56,-2.89,-3.44,-2.81,-2.81,-2.77,-2.92,-2.81,-2.88,-3.02,-2.99,-2.31,-2.59,-3.15,-2.87,21,83.133333,275.800000
17,PriceChange,18,90.91,77.27,90.91,81.82,77.27,77.27,81.82,90.91,81.82,90.91,81.82,77.27,77.27,81.82,86.36,81.82,72.73,81.82,77.27,81.82,81.82,81.82,72.73,72.73,-1.82,-1.74,-2.40,-1.99,-1.60,-2.01,-2.05,-1.88,-1.92,-2.23,-2.77,-2.51,-1.85,-2.35,-2.77,-2.08,-0.94,-1.47,-2.06,-2.35,-2.25,-2.44,-2.24,-2.22,22,81.250417,247.509167
18,PriceChange,19,90.91,77.27,90.91,81.82,77.27,77.27,81.82,90.91,81.82,90.91,81.82,77.27,77.27,81.82,86.36,81.82,72.73,81.82,77.27,81.82,81.82,81.82,72.73,72.73,-1.82,-1.74,-2.40,-1.99,-1.60,-2.01,-2.05,-1.88,-1.92,-2.23,-2.77,-2.51,-1.85,-2.35,-2.77,-2.08,-0.94,-1.47,-2.06,-2.35,-2.25,-2.44,-2.24,-2.22,22,81.250417,247.509167
11,PriceChange,12,92.86,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,92.86,92.86,92.86,92.86,85.71,92.86,85.71,78.57,85.71,85.71,85.71,85.71,85.71,78.57,85.71,-2.59,-2.40,-2.86,-2.09,-2.61,-2.72,-2.21,-2.14,-2.64,-3.32,-3.57,-3.48,-3.32,-2.88,-3.12,-3.39,-3.05,-3.16,-3.62,-3.34,-2.52,-4.58,-4.61,-4.62,14,86.307500,228.305000
12,PriceChange,13,92.86,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,92.86,92.86,92.86,92.86,85.71,92.86,85.71,78.57,85.71,85.71,85.71,85.71,85.71,78.57,85.71,-2.59,-2.40,-2.86,-2.09,-2.61,-2.72,-2.21,-2.14,-2.64,-3.32,-3.57,-3.48,-3.32,-2.88,-3.12,-3.39,-3.05,-3.16,-3.62,-3.34,-2.52,-4.58,-4.61,-4.62,14,86.307500,228.305000
13,PriceChange,14,92.86,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,92.86,92.86,92.86,92.86,85.71,92.86,85.71,78.57,85.71,85.71,85.71,85.71,85.71,78.57,85.71,-2.59,-2.40,-2.86,-2.09,-2.61,-2.72,-2.21,-2.14,-2.64,-3.32,-3.57,-3.48,-3.32,-2.88,-3.12,-3.39,-3.05,-3.16,-3.62,-3.34,-2.52,-4.58,-4.61,-4.62,14,86.307500,228.305000
14,PriceChange,15,93.75,68.75,87.50,87.50,81.25,87.50,87.50,87.50,81.25,87.50,87.50,87.50,87.50,81.25,87.50,81.25,75.00,81.25,81.25,81.25,81.25,81.25,75.00,81.25,-2.41,-2.40,-2.65,-1.99,-1.60,-2.39,-2.05,-1.88,-1.92,-2.73,-2.98,-3.48,-3.32,-2.88,-3.12,-3.39,-3.05,-3.16,-3.62,-3.34,-2.52,-4.06,-4.21,-4.19,16,83.333333,213.333333
15,PriceChange,16,94.12,70.59,88.24,82.35,76.47,82.35,82.35,88.24,82.35,88.24,82.35,82.35,82.35,82.35,88.24,82.35,76.47,82.35,82.35,82.35,82.35,82.35,70.59,76.47,-2.31,-2.24,-2.62,-1.96,-1.58,-2.07,-2.03,-1.80,-1.84,-2.56,-2.89,-3.44,-2.81,-2.81,-2.77,-2.92,-2.81,-2.88,-3.02,-2.99,-2.31,-3.47,-3.44,-3.65,17,82.107083,205.820

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [15]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)